<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D3_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 데이터 준비
df = pd.read_csv('/content/Fish.csv')

In [12]:
df2 = df[['Length1', 'Height', 'Width', 'Weight', 'Species']]
df3 = df2.loc[df2['Species'] == 'Perch']

In [ ]:
# 선형회귀 -> 독립변수(원인)로 종속변수(결과) 예측
# 다중회귀 -> 독립변수가 여러개인 것. ex) 독립 - 길이, 높이, 너비 , 종속 - 무게 
# 다항회귀 -> 1차식으로는 설명이 안되는 복잡한 데이터를 설명하기 위해 차수를 높이는 방법.(다중회귀)

In [ ]:
from sklearn.model_selection import train_test_split
trd, tsd, trt, tst = train_test_split(df3[['Length1']], df3['Weight'],train_size = 0.7, random_state=42)
trd

In [17]:
from sklearn.linear_model import LinearRegression

## 일반 선형회귀
lr = LinearRegression()
lr.fit(trd, trt)

# 모델이 학습데이터에 대한 점수 낮고, 테스트 데이터에 대한 점수가 낮은 경우 -> 과소적합
# 모델이 학습 데이터에 대한 점수가 매우 높은데, 테스트 점수는 낮은 경우 -> 과대 적합

lr.score(trd, trt), lr.score(tsd, tst) # 약간의 과대적합으로 보임



(0.9345639427973997, 0.8396353302211251)

In [33]:
## 다항회귀 + 다중회귀
## 다중 회귀 적용 (길이, 높이, 너비)

mtrd, mtsd, mtrt, mtst = train_test_split(df3[['Length1', 'Height', 'Width']], df3['Weight'],train_size = 0.7, random_state=42)

# 단일 선형회귀의 회귀식 모양
# a * 길이 + b

# 다중 선형회귀의 회귀식 모양
# 회귀계수 a1,a2,a3 -> 가중치
# a1 * 길이 + a2 * 높이 + a3 * 너비 + b


## 다중 회귀 모델로 예측
lr2 = LinearRegression()
lr2.fit(mtrd, mtrt)

lr2.score(mtrd, mtrt), lr2.score(mtsd, mtst)


(0.9532592691768272, 0.8894875299277586)

In [ ]:
print(lr.coef_, lr.intercept_)
print(lr2.coef_, lr2.intercept_)

In [ ]:
# 표준화 -> 서로 다른 분포를 가진 수치를 한 기준으로 통일하는 것
from sklearn.preprocessing import StandardScaler

## 변환기 -> 데이터 변형에 사용
## 추정기 -> 데이터 예측에 사용
ss = StandardScaler() # 표준화 변환기

## 변환기 사용법
## fit - 변환에 필요한 데이터 세팅
## transform - 변환 수행

ss.fit(mtrd, mtrt) # 데이터 세팅
scaled_mtrd = ss.transform(mtrd) 
scaled_mtsd = ss.transform(mtsd)


## 표준화된 데이터로 학습한 모델
lr3 = LinearRegression()
lr3.fit(scaled_mtrd, mtrt)

lr3.score(scaled_mtrd, mtrt), lr3.score(scaled_mtsd, mtst)


In [64]:
# 기존의 특성(feature)을 바탕으로 새로운 특성을 만들어낼 수 있다.
## 특성 공학(feature engeering)

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=5) # degree를 이용해 차수를 조정할 수 있다.

poly.fit(scaled_mtrd, mtrt) 
poly_scaled_mtrd5 = poly.transform(scaled_mtrd)
poly_scaled_mtsd5 = poly.transform(scaled_mtsd)

#poly.get_feature_names_out() # 기존의 특성을 이용해 차수 5까지 새로운 특성을 조합해냄



In [ ]:
## 특성 공학을 적용한 모델
lr4 = LinearRegression()
lr4.fit(poly_scaled_mtrd5, mtrt)

## 학습데이터 점수는 매우 높고, 테스트 데이터 점수는 낮게 나오는 경우 -> 과대적합
lr4.score(poly_scaled_mtrd5, mtrt), lr4.score(poly_scaled_mtsd5, mtst)

In [ ]:
## 과소적합 해결 -> 더 많은 데이터를 학습. 데이터를 더 복잡하게 만듦
## 과대적합 해결 -> 규제를 가한다.

## 규제가 적용된 회귀 모델 
## 1. 릿지 -> 특성이 대체적으로 비슷한 경우, 일반적으로 많이 씀
## 2. 라쏘 -> 중요한 특성이 확실히 보이는 경우.


from sklearn.linear_model import Ridge
ridge = Ridge()

ridge.fit(poly_scaled_mtrd5, mtrt)
ridge.score(poly_scaled_mtrd5, mtrt), ridge.score(poly_scaled_mtsd5, mtst)

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

for alpha in alpha_list :
  ridge.alpha = alpha
  
  ridge.fit(poly_scaled_mtrd5, mtrt)
  print(ridge.score(poly_scaled_mtrd5, mtrt), ridge.score(poly_scaled_mtsd5, mtst))

## 하이퍼 파라미터 alpha



In [70]:
# 기존의 특성(feature)을 바탕으로 새로운 특성을 만들어낼 수 있다.
## 특성 공학(feature engeering)

from sklearn.preprocessing import PolynomialFeatures
poly2 = PolynomialFeatures(degree=2) # degree를 이용해 차수를 조정할 수 있다.

poly2.fit(scaled_mtrd, mtrt) 
poly_scaled_mtrd2 = poly2.transform(scaled_mtrd)
poly_scaled_mtsd2 = poly2.transform(scaled_mtsd)

#poly2.get_feature_names_out()

array(['1', 'x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()

ridge.fit(poly_scaled_mtrd2, mtrt)
ridge.score(poly_scaled_mtrd2, mtrt), ridge.score(poly_scaled_mtsd2, mtst)

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

for alpha in alpha_list :
  ridge.alpha = alpha
  
  ridge.fit(poly_scaled_mtrd2, mtrt)
  print(ridge.score(poly_scaled_mtrd2, mtrt), ridge.score(poly_scaled_mtsd2, mtst))

## 하이퍼 파라미터 alpha


In [ ]:
ridge2 = Ridge(alpha=0.1)
ridge2.fit(poly_scaled_mtrd2, mtrt)

In [ ]:
# 라쏘를 이용해서 차수가 3인 학습 데이터를 학습시키고 평가하기
from sklearn.linear_model import Lasso

df2[['Length1',	'Height',	'Width']]
df2['Weight']

# 쪼개고
mtrd, mtsd, mtrt, mtst = train_test_split(df2[['Length1',	'Height',	'Width']], df2['Weight'], train_size=0.7, random_state=42)

# 표준화
ss = StandardScaler()

ss.fit(mtrd)
scaled_mtrd = ss.transform(mtrd)
scaled_mtsd = ss.transform(mtsd)

# 피처 엔지니어링

poly3 = PolynomialFeatures(degree=3)

poly3.fit(scaled_mtrd)
poly_scaled_mtrd3 = poly3.transform(scaled_mtrd)
poly_scaled_mtsd3 = poly3.transform(scaled_mtsd)

lasso = Lasso()
lasso.fit(poly_scaled_mtrd3, mtrt)
lasso.score(poly_scaled_mtrd3, mtrt), lasso.score(poly_scaled_mtsd3, mtst)